In [4]:
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModel
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
import evaluate
import numpy as np
import pandas as pd
import os
from models import columns,vectorize_data
from resultsAnalyse import drawConfusionMatrix
import torch
from sklearn.metrics import f1_score
def filter_inadequada(example):
    return example["INADEQUADA"] == 0

ds = load_dataset("higopires/RePro-categories-multilabel")
print(len(ds["train"]))
#remove INADEQUADA examples
ds = ds.filter(filter_inadequada)
#remove only keep the firs 1000 examples
#ds["train"] = ds["train"].filter(lambda example, idx: idx < 20, with_indices=True)
#ds["test"] = ds["test"].filter(lambda example, idx: idx < 5, with_indices=True)
#ds["validation"] = ds["validation"].filter(lambda example, idx: idx < 5, with_indices=True)
print(len(ds["train"]))

8002
7674


In [5]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Convert logits to probabilities and then to binary predictions
    predictions = (torch.sigmoid(torch.tensor(logits)) > 0.5).int().numpy()
    #print(f"DEBUG: Original logits shape: {logits.shape}")
    #print(f"DEBUG: Original labels shape: {labels.shape}")
    # Calculate sample-wise F1 score
    f1 = f1_score(labels, predictions, average='micro', zero_division=0)
    
    return {'f1_micro': float(f1)}
#model_name = "neuralmind/bert-large-portuguese-cased"
model_name = "Qwen/Qwen2.5-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
def preprocess_function(sample):
    # Tokenize text
    tokenized = tokenizer(sample["review_text"], truncation=True, padding=True, max_length=512, return_tensors="pt")
    
    # Convert binary label columns to a list (e.g., [1, 0, 1, 0, 0, 0])
    labels = []
    for i in range(len(sample["review_text"])):
        label_row = [
            float(sample["ENTREGA"][i]),
            float(sample["OUTROS"][i]),
            float(sample["PRODUTO"][i]),
            float(sample["CONDICOESDERECEBIMENTO"][i]),
            float(sample["ANUNCIO"][i])
        ]
        labels.append(label_row)
    
    tokenized["labels"] = torch.tensor(labels, dtype=torch.float)
    return tokenized
#check if output layer has 5 outputs
#print(model.classifier.out_features)
#model.classifier.out_features = 5  # Explicitly ensure final layer has 5 outputs

Add the LORA

In [6]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5,problem_type="multi_label_classification",ignore_mismatched_sizes=True)
#print(model.classifier)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of Qwen2ForSequenceClassification were not initialized from the model checkpoint at Qwen/Qwen2.5-3B-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
from peft import LoraConfig, TaskType
from peft import get_peft_model
peft_config = LoraConfig(task_type=TaskType.SEQ_CLS #for sequence classification
                         , inference_mode=False #for inference mode
                         , r=8, lora_alpha=32, lora_dropout=0.1) #for dropout

In [ ]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()
#unfreeze the classifier layer
#for param in model.base_model.model.classifier.parameters():
#   param.requires_grad = True  # Make this layer trainable
#model.print_trainable_parameters()
device = "cuda" if torch.cuda.is_available() else "cpu"

trainable params: 1,853,440 || all params: 3,087,802,368 || trainable%: 0.0600
trainable params: 1,853,440 || all params: 3,087,802,368 || trainable%: 0.0600


C:\Users\tomas\AppData\Roaming\Python\Python39\site-packages\peft\mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
C:\Users\tomas\AppData\Roaming\Python\Python39\site-packages\peft\mapping_func.py:79: UserWarning: The PEFT config's `base_model_name_or_path` was renamed from 'Qwen/Qwen2.5-3B-Instruct' to 'None'. Please ensure that the correct base model is loaded when loading this checkpoint.
  warnings.warn(
C:\Users\tomas\AppData\Roaming\Python\Python39\site-packages\peft\tuners\tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [10]:
model.to(device)
tokenized_dataset = ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/7674 [00:00<?, ? examples/s]

Map:   0%|          | 0/952 [00:00<?, ? examples/s]

Map:   0%|          | 0/966 [00:00<?, ? examples/s]

In [11]:
training_args = TrainingArguments(
    output_dir="./resultsTransformer",
    learning_rate=1e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=10,
    weight_decay=0.01,
    eval_strategy="epoch", # run validation at the end of each epoch
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_micro",

)
tokenizer = AutoTokenizer.from_pretrained(model_name)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,

    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


C:\Users\tomas\AppData\Local\Temp\ipykernel_23344\2879474932.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [12]:
trainer.train()

TypeError: forward() got an unexpected keyword argument 'num_items_in_batch'

In [ ]:
y_pred = trainer.predict(tokenized_dataset["test"])
y_pred.metrics

{'test_loss': 0.6443551778793335,
 'test_f1_micro': 0.30769230769230765,
 'test_runtime': 0.1391,
 'test_samples_per_second': 35.952,
 'test_steps_per_second': 7.19}

In [ ]:
trainer.save_model("lora_Portuguese_model")

In [ ]:
del model  # Remove Python reference
torch.cuda.empty_cache()  # Clear GPU memory